# Epicurious Recipe Webscrape

Guide to Web Scraping with Python Part 1: Requests and BeautifulSoup
https://www.learndatasci.com/tutorials/ultimate-guide-web-scraping-w-python-requests-and-beautifulsoup/

In [1]:
from bs4 import BeautifulSoup
import time
from time import sleep
import requests
import json
import re

# Scrape recipe urls from a list of categories

In [22]:
# https://www.epicurious.com/services/sitemap
# https://www.epicurious.com/services/sitemap/recipes/member/2019
member_url = "https://www.epicurious.com/services/sitemap/recipes/member/"
year = "2019"
member_year_url = member_url + year
member_year_url

'https://www.epicurious.com/services/sitemap/recipes/member/2019'

In [91]:
# get the recipe urls from the first page
r = requests.get(member_year_url)

# Save HTML locally
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)
        
save_html(r.content, 'recipe_list.html')

# Read local HTML file
def open_html(path):
    with open(path, 'rb') as f:
        return f.read()
    
    
html = open_html('recipe_list.html')
soup = BeautifulSoup(r.content, 'html.parser')

In [128]:
# this only returns the first value ... 
# need to loop and save all values 
# then click to next page 

recipe_list_url = "https://epicurious.com"+soup.find('div', id="sitemapItems").a['href']
recipe_list_title = soup.find('div', id="sitemapItems").a.text

recipe_list_url # 'https://epicurious.com/recipes/member/views/omelet-benedict-5e08f2f054b6560009f82294'
recipe_list_title # 'Omelet Benedict'

'Omelet Benedict'

# Scrape recipe page, save HTML locally, read local HTML (DONE)

In [ ]:
### Do you want to save all of the scraped HTML files?

In [61]:
# Visit specific recipe website >> add time.sleep(1) if site is loading slowly

base = "https://epicurious.com/recipes"
url = "/food/views/overnight-porridge-congee-chao-andrea-nguyen-vietnamese-rice-soup/"

r = requests.get(base+url)

# Save HTML locally
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)
        
save_html(r.content, 'recipe.html')

# Read local HTML file
def open_html(path):
    with open(path, 'rb') as f:
        return f.read()
    
    
html = open_html('recipe.html')

In [62]:
# Soupify
soup = BeautifulSoup(r.content, 'html.parser')

# Scrape the recipe for general information (DONE)
(not scraped: special equipment, preparation, menus, related content)

In [ ]:
### do you want to scrape any of these sections above?
# write a functionfor each variable pull

In [63]:
recipe = dict()  
recipe['title'] = soup.find('div', class_='title-source').h1.text                 # string
recipe['author'] = soup.find(class_='contributor')['title']                       # string
date = soup.find(class_="pub-date").text.split()                                  # string
recipe['month'] = date[0]                                                         # string
recipe['year'] = int(date[1])                                                     # int
recipe['rating'] = float(soup.find(class_="user-interactions").meta['content'])   # float
recipe['reviews'] = int(soup.find(class_="reviews-count").text)                   # int
recipe['make_again'] = soup.find('div', class_="prepare-again-rating").span.text  # int (float?)
recipe['active_time'] = soup.find('dd', class_="active-time").text                # string

In [64]:
recipe

{'title': 'Super-Simple Overnight Porridge ',
 'author': 'Andrea Nguyen',
 'month': 'November',
 'year': 2019,
 'rating': 4.0,
 'reviews': 1,
 'make_again': '100%',
 'active_time': '30 minutes, plus overnight soaking'}

# Ingredients (DONE)

In [67]:
ingredients = []

lis = soup.find_all('li', class_="ingredient")

for li in lis:
    newsoup = BeautifulSoup(str(li), 'html.parser')
    ingredients.append(li.text)
    
recipe['ingredients'] = ingredients  # list of strings
ingredients

['2 cups packed cooked white rice',
 'About 5 cups chicken stock, vegetable stock, or store-bought chicken or vegetable broth',
 '2 cups water, plus more as needed',
 '3 thick slices unpeeled ginger, bruised',
 '2 green onions, white parts kept whole, green parts cut into thin rings',
 'About ½ teaspoon fine sea salt',
 'Recently ground black pepper (optional)']

In [58]:
recipe['ingr_len'] = len(ingredients)  # int

# Nutritional Information

In [13]:
nutrition = dict()

nutrition['cal'] = int(soup.find('span', class_="nutri-data", itemprop="calories").text)
nutrition['carb'] = (soup.find('span', class_="nutri-data", itemprop="carbohydrateContent").text)
nutrition['fat'] = soup.find('span', class_="nutri-data", itemprop="fatContent").text
nutrition['protein'] = soup.find('span', class_="nutri-data", itemprop="proteinContent").text
nutrition['sat_fat'] = soup.find('span', class_="nutri-data", itemprop="saturatedFatContent").text
nutrition['sodium'] = soup.find('span', class_="nutri-data", itemprop="sodiumContent").text
nutrition['polyunsat_fat'] = soup.findAll('span', class_="nutri-data")[6].text # no attr = itemprop
nutrition['fiber'] = soup.find('span', class_="nutri-data", itemprop="fiberContent").text
nutrition['monounsat_fat'] = soup.findAll('span', class_="nutri-data")[8].text # no attr = itemprop
nutrition['cholesterol'] = soup.findAll('span', class_="nutri-data")[9].text # no attr = itemprop
nutrition['servings'] = soup.find(class_="per-serving").text

recipe['nutrition'] = nutrition # dict of dict [int, int]
nutrition

{'cal': 127,
 'carb': '28 g(9%)',
 'fat': '0 g(1%)',
 'protein': '2 g(5%)',
 'sat_fat': '0 g(0%)',
 'sodium': '342 mg(14%)',
 'polyunsat_fat': '0 g',
 'fiber': '1 g(3%)',
 'monounsat_fat': '0 g',
 'cholesterol': '',
 'servings': 'per serving (4 servings)'}

In [14]:
carb = re.findall('\d+', (soup.find('span', class_="nutri-data", itemprop="carbohydrateContent").text))
carb

['28', '9']

# Tags

In [ ]:
### What is the best way to save this data in a format that preserves the category and tag relationship?

In [ ]:
tags = []
tags = soup.find('dl', class_='tags').a['href'].split("/")[1:]
tags = {"category" : tags[0], "tag" : tags[1]}
tags

# The whole shabang (DONE)

In [15]:
recipe

{'title': 'Super-Simple Overnight Porridge ',
 'author': 'Andrea Nguyen',
 'month': 'November',
 'year': 2019,
 'rating': 4.0,
 'reviews': 1,
 'make_again': '100%',
 'active_time': '30 minutes, plus overnight soaking',
 'ingredients': ['2 cups packed cooked white rice',
  'About 5 cups chicken stock, vegetable stock, or store-bought chicken or vegetable broth',
  '2 cups water, plus more as needed',
  '3 thick slices unpeeled ginger, bruised',
  '2 green onions, white parts kept whole, green parts cut into thin rings',
  'About ½ teaspoon fine sea salt',
  'Recently ground black pepper (optional)'],
 'ingr_len': 7,
 'nutrition': {'cal': 127,
  'carb': '28 g(9%)',
  'fat': '0 g(1%)',
  'protein': '2 g(5%)',
  'sat_fat': '0 g(0%)',
  'sodium': '342 mg(14%)',
  'polyunsat_fat': '0 g',
  'fiber': '1 g(3%)',
  'monounsat_fat': '0 g',
  'cholesterol': '',
  'servings': 'per serving (4 servings)'}}

# Save recipe as a JSON (DONE)

In [16]:
all_recipes = []
with open('recipe.json', 'w') as f:
    json.dump(recipe, f)

# Save multiple recipes to the category search JSON

In [ ]:
### Need to decide how to organize this information

In [ ]:
all_recipes.append(recipe)
with open('all_recipe.json', 'w') as f:
    json.dump(all_recipes, f)